#Arbitrated Dynamic Ensemble

##Get Data

In [ ]:
#install libraries
!pip install --upgrade linear-tree

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from google.colab import drive
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import shap
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
from joblib import Parallel, delayed, dump, load
import xgboost as xgb
import time
import yfinance as yf
import torch.nn.functional as F
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from lineartree import LinearBoostRegressor, LinearBoostClassifier
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge


#ignore warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#global formatting for the plots
plt.rcParams.update({
    'axes.titlesize': 18,    # Title font size
    'axes.labelsize': 14,    # x/y label font size
    'xtick.labelsize': 12,   # x-axis tick label size
    'ytick.labelsize': 12,   # y-axis tick label size
    'legend.fontsize': 12,   # Legend text size
    'legend.title_fontsize': 13  # Legend title size
})

In [ ]:
drive.mount('/content/gdrive')

#import time series
datasets = {}
for name in ['sp500', 'eur_usd']:
    datasets[name] = pd.read_csv(f'/content/gdrive/MyDrive/università/Machine_learning/Project_code/Datasets/{name}_data.csv', index_col=0)
    datasets[name] = datasets[name]
    datasets[name].index.name = 'Date'
    datasets[name].index = pd.to_datetime(datasets[name].index)


Close_price = {}
Log_return = {}
test_residuals = {}
val_residuals = {}
SARIMA_fitted_values = {}
SARIMA_test_predictions = {}
residuals = {}

for name , dataset in datasets.items():

    #save Close price and Log Returns into a pd.Series
    Close_price[name] = dataset['Close'].copy().astype(float)
    # Log_return[name] = dataset['Log Return'].copy().astype(float)

    #drop columns not needed
    # dataset.drop(['Open', 'High', 'Low', 'Close', 'Log Return'], axis=1, inplace=True)
    dataset.drop(['Open', 'High', 'Low', 'Close'], axis=1, inplace=True)


    #import SARIMA residuals
    residuals[name] = pd.read_csv(f'/content/gdrive/MyDrive/università/Machine_learning/Project_code/ARIMA/{name}/SARIMA_residuals.csv', index_col=0, sep=",")
    residuals[name].index = pd.to_datetime(residuals[name].index)


    #val residuals
    val_residuals[name] = pd.read_csv(f'/content/gdrive/MyDrive/università/Machine_learning/Project_code/ARIMA/{name}/SARIMA_val_residuals.csv', index_col=0, sep=",")
    val_residuals[name].index = pd.to_datetime(val_residuals[name].index)

    #test residuals
    test_residuals[name] = pd.read_csv(f'/content/gdrive/MyDrive/università/Machine_learning/Project_code/ARIMA/{name}/SARIMA_test_residuals.csv', index_col=0, sep=",")
    test_residuals[name].index = pd.to_datetime(test_residuals[name].index)

    #import SARIMA fitted values (predictions on the training set)
    SARIMA_fitted_values[name] = pd.read_csv(f'/content/gdrive/MyDrive/università/Machine_learning/Project_code/ARIMA/{name}/SARIMA_fitted_values.csv', index_col=0)
    SARIMA_fitted_values[name].index = pd.to_datetime(SARIMA_fitted_values[name].index)

    #import SARIMA predictions on the test set
    SARIMA_test_predictions[name] = pd.read_csv(f'/content/gdrive/MyDrive/università/Machine_learning/Project_code/ARIMA/{name}/SARIMA_test_predictions.csv', index_col=0).squeeze() #to read as a pd.Series
    SARIMA_test_predictions[name].index = pd.to_datetime(SARIMA_test_predictions[name].index)

Mounted at /content/gdrive


###Helper functions to move data to the GPU

In [ ]:
#define some helper classes
def get_device():
    if torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'

    return device

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    elif isinstance(data, torch.Tensor):  # Only move tensors to the device
        return data.to(device, non_blocking=True)
    else:
        return data  # For non-tensor types (e.g., strings), return as is


class DeviceDataLoader (): #receive a dataloader and move to the correct device
  def __init__(self, dl, device):
    self.dl = dl
    self.device = device

  def __iter__(self):
    for batch in self.dl:
      yield to_device(batch, device)

  def __len__(self):
    return len(self.dl)


#get device
device = get_device()

###Get pre-trained models

LSTM followed by CNN

In [ ]:
#custom loss
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))


#istance of the custom loss
rmse_loss = torch.nn.MSELoss().to(device) #the loss is computed off the mse loss, the function return the rmse for easier comprehension


class LSTM_CNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__() #initialize the parent class
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True) #This argument specifies the input and output tensors are provided as (batch, seq, feature)
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=hidden_size, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            #flatten
            nn.Flatten(),
            nn.LazyLinear(out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=num_classes)
        )

    def forward(self, x):
        out, _ = self.lstm(x)  #stateless LSTM each batch is using a different hidden/cell state (initialized to 0)
        out = out.permute(0, 2, 1)
        out = self.cnn(out)
        # out = out.squeeze(-1)
        # print(out)
        return out

    def training_step(self, batch):
        x, y = batch
        out = self(x) # This calls self.forward(x) through the __call__ method
        #loss = torch.sqrt(F.mse_loss(out, y)) incorrect way
        loss = rmse_loss(out, y)
        return loss

    def validation_step(self, batch):
        x, y = batch
        out = self(x)
        #loss = torch.sqrt(F.mse_loss(out, y))
        loss = rmse_loss(out, y)
        return {'val_loss': loss.detach()} #loss.detatch disable gradient computation

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss']))


#instance of the model
LSTM_CNN_model = {}

for name, dataset in datasets.items():

    num_classes = 1 # regression
    # input_size = 32  # Number of features per time step
    input_size = 1 #number of input features
    hidden_size = 512 #number of hidden layer in each cell, the more is better, but also will slow down the training
    num_layers = 1

    LSTM_CNN_model[name] = LSTM_CNN(num_classes= num_classes, input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)
    LSTM_CNN_model[name].to(device) #move the istance to the device

CNN + LSTM parallel architecture

In [ ]:
class ParallelCNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(out_features=128), #linear layer that automatically infer the input size
            nn.ReLU()
        )
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc_lstm = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128*2, num_classes)

    def forward(self, x):
        #cnn takes input of shape (batch_size, channels, seq_len)
        x_cnn = x.permute(0, 2, 1)
        out_cnn = self.cnn(x_cnn)
        # lstm takes input of shape (batch_size, seq_len, input_size)
        out_lstm, _ = self.lstm(x)
        out_lstm = self.fc_lstm(out_lstm[:, -1, :])
        out = torch.cat([out_cnn, out_lstm], dim=1)
        out = self.fc(out)
        return out

    def training_step(self, batch):
        x, y = batch
        out = self(x) # This calls self.forward(x) through the __call__ method
        #loss = torch.sqrt(F.mse_loss(out, y)) incorrect way
        loss = rmse_loss(out, y)
        return loss

    def validation_step(self, batch):
        x, y = batch
        out = self(x)
        #loss = torch.sqrt(F.mse_loss(out, y))
        loss = rmse_loss(out, y)
        return {'val_loss': loss.detach()} #loss.detatch disable gradient computation

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss']))


#instance of the model
parallelCNNLSTM_model = {}

for name, dataset in datasets.items():

    num_classes = 1 # regression
    # input_size = 32  # Number of features per time step
    input_size = 1 #number of input features
    hidden_size = 512 #number of hidden layer in each cell, the more is better, but also will slow down the training
    num_layers = 1

    parallelCNNLSTM_model[name] = ParallelCNN_LSTM(num_classes= num_classes, input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)
    parallelCNNLSTM_model[name].to(device) #move the istance to the device

LSTM

In [ ]:
#custom loss
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))


#istance of the custom loss
rmse_loss = torch.nn.MSELoss().to(device) #the loss is computed off the mse loss, the function return the rmse for easier comprehension

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size=512, num_layers=3, dropout=0.3):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes

        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, self.num_classes)

        # Activation and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)

        # Pass only the last output
        lstm_out = lstm_out[:, -1, :]

        # Fully connected layers with ReLU and Dropout
        x = self.relu(self.fc1(lstm_out))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)  # Final prediction
        return x

    def training_step(self, batch):
        x, y = batch
        out = self(x) # This calls self.forward(x) through the __call__ method
        #loss = torch.sqrt(F.mse_loss(out, y)) incorrect way
        loss = rmse_loss(out, y)
        return loss

    def validation_step(self, batch):
        x, y = batch
        out = self(x)
        #loss = torch.sqrt(F.mse_loss(out, y))
        loss = rmse_loss(out, y)
        return {'val_loss': loss.detach()} #loss.detatch disable gradient computation

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss']))


#instance of the model
LSTM_model = {}

for name, dataset in datasets.items():

    num_classes = 1 # regression
    # input_size = 32  # Number of features per time step
    input_size = 1 #number of input features
    hidden_size = 512 #number of hidden layer in each cell, the more is better, but also will slow down the training
    num_layers = 3

    LSTM_model[name] = LSTM(num_classes= num_classes, input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)
    LSTM_model[name].to(device) #move the istance to the device

CNN model

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(out_features=128), #linear layer that automatically infer the input size
            nn.ReLU()
        )
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        #cnn takes input of shape (batch_size, channels, seq_len)
        x_cnn = x.permute(0, 2, 1)
        out_cnn = self.cnn(x_cnn)
        out = self.fc(out_cnn)
        return out

    def training_step(self, batch):
        x, y = batch
        out = self(x) # This calls self.forward(x) through the __call__ method
        #loss = torch.sqrt(F.mse_loss(out, y)) incorrect way
        loss = rmse_loss(out, y)
        return loss

    def validation_step(self, batch):
        x, y = batch
        out = self(x)
        #loss = torch.sqrt(F.mse_loss(out, y))
        loss = rmse_loss(out, y)
        return {'val_loss': loss.detach()} #loss.detatch disable gradient computation

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss']))


#instance of the model
CNN_model = {}

for name, dataset in datasets.items():

    num_classes = 1 # regression
    # input_size = 32  # Number of features per time step
    input_size = 1 #number of input features
    hidden_size = 512 #number of hidden layer in each cell, the more is better, but also will slow down the training
    num_layers = 1

    CNN_model[name] = CNN(num_classes= num_classes, input_size = input_size, hidden_size = hidden_size, num_layers = num_layers)
    CNN_model[name].to(device) #move the istance to the device

Load models


In [ ]:
# from inspect import modulesbyfile
drive.mount("/content/gdrive")

Linear_booster_model = {}
#import trained models
model_names = ["Linear_booster.joblib", "CNN", "LSTM", "LSTM_CNN", "ParallelCNN_LSTM"]
model_istances = ["Linear_booster_model", CNN_model, LSTM_model, LSTM_CNN_model, parallelCNNLSTM_model]


for name, _ in datasets.items():
    for i, model in enumerate(model_names):
        if model == "Linear_booster.joblib":
            model_path = f'/content/gdrive/MyDrive/università/Tesi/models/{name}/{model}'
            Linear_booster_model[name] = load(model_path)
        else:
            #load state dict
            model_path = f'/content/gdrive/MyDrive/università/Tesi/models/{name}/{model}_fine_tuned'
            model_istances[i][name].load_state_dict(torch.load(model_path, map_location=torch.device(device)))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##ADE library

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from google.colab import drive
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from joblib import Parallel, delayed, dump, load
import xgboost as xgb
import time
import yfinance as yf
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor
import threading

#suppress warnings
import warnings
warnings.filterwarnings('ignore')


class OverlappingTimeSeriesSplit():
    def __init__(self, n_splits=5, window_size=10):
        self.n_splits = n_splits
        self.window_size = window_size

    def split(self, X):
        n_samples = len(X)
        fold_size = n_samples // (self.n_splits + 1)  # Ensure proper splitting

        for i in range(1, self.n_splits + 1):  # Start from 1 to avoid empty train set
            train_end = i * fold_size
            train_idx = range(train_end)  # Train set includes all data before validation

            val_start = max(0, train_end - self.window_size)
            val_end = train_end + fold_size
            val_idx = range(val_start, min(val_end, n_samples))  # Ensure within bounds

            yield list(train_idx), list(val_idx)


def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    smape_value = np.mean(numerator / denominator)  # Renamed variable to avoid conflict
    return smape_value

#deep learning meta model architecture
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size=512, num_layers=3, dropout=0.3):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes

        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, self.num_classes)

        # Activation and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)

        # Pass only the last output
        lstm_out = lstm_out[:, -1, :]

        # Fully connected layers with ReLU and Dropout
        x = self.relu(self.fc1(lstm_out))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)  # Final prediction
        return x


#deep learning meta model architecture
class ParallelCNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(out_features=128), #linear layer that automatically infer the input size
            nn.ReLU()
        )
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc_lstm = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128*2, num_classes)

    def forward(self, x):
        #cnn takes input of shape (batch_size, channels, seq_len)
        x_cnn = x.permute(0, 2, 1)
        out_cnn = self.cnn(x_cnn)
        # lstm takes input of shape (batch_size, seq_len, input_size)
        out_lstm, _ = self.lstm(x)
        out_lstm = self.fc_lstm(out_lstm[:, -1, :])
        out = torch.cat([out_cnn, out_lstm], dim=1)
        out = self.fc(out)
        return out


class Custom_df(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index): #this method allows to retrieve a specific sample from the dataset based on its index (the index is passed to this method)
        input = torch.tensor(np.array(self.x[index]), dtype=torch.float32) #convert np array into a tensor
        target = torch.tensor(np.array(self.y[index]), dtype=torch.float32)
        return input, target

    def __len__(self):
        return len(self.x)


class DeviceDataLoader (): #receive a dataloader and move to the correct device
  def __init__(self, dl : DataLoader, device):
    self.dl = dl
    self.device = device

  def __iter__(self):
    for batch in iter(self.dl): #iter will reset the iterator every time the __iter__ mathod gets called
      yield ADE.to_device(batch, device)

  def __len__(self):
    return len(self.dl)



class ADE ():
    def __init__(self, trained_models: list, meta_model, train_data: pd.DataFrame, test_data: pd.DataFrame, window_size : int = 10, error_window = 10, k=3, temperature=1, stacking_preds = None):
        self.trained_models = trained_models
        self.meta_model = self.get_meta_model(meta_model) #accept a string as an input
        self.train_data = pd.DataFrame(train_data)
        self.test_data = pd.DataFrame(test_data)
        self.MinMaxscaler = MinMaxScaler(feature_range=(-1, 1))
        self.Robustscaler = RobustScaler()
        # self.error_scaler = MinMaxScaler(feature_range=(-1, 1))
        self.window_size = window_size
        self.error_window = error_window
        self.k = k
        self.temperature = temperature
        self.test_predictions = []
        self.test_error_predictions = []
        self.device = self.get_device()
        self.lock = threading.Lock()
        self.ensemble_preds = None
        self.tscv = OverlappingTimeSeriesSplit(n_splits=5, window_size=window_size)
        self.oof_predictions = []
        self.first_fold_idx = None
        self.target_scaler = RobustScaler()
        self.stacking_preds = stacking_preds


        #if there are deep learning models
        if any(isinstance(model, torch.nn.Module) for model in trained_models):
            self.train_loader, self.test_loader = self.data_preprocessing_deep_learning(self.train_data, self.test_data, self.MinMaxscaler, self.window_size)

        #if there are machine learning models
        if any(not isinstance(model, torch.nn.Module) for model in trained_models):
            self.X_train, self.y_train, self.X_test, self.y_test = self.data_preprocessing_machine_learning(self.train_data, self.test_data, self.Robustscaler, self.window_size)


    def feature_engineer(self, oof_predictions, real_data):
        # print("feature engineering")
        df = pd.DataFrame(index = real_data.index)
        oof_predictions = pd.DataFrame(oof_predictions, index = real_data.index)

        #compute errors
        ae_errors = np.abs(real_data.iloc[:,0] - oof_predictions.iloc[:,0])

        # print("ae errors")
        # print(ae_errors.head())

        #compute rolling MAE
        rolling_mae = np.zeros(len(oof_predictions))  # Assuming oof_predictions is 1D

        for i in range(len(oof_predictions)):
            start = max(0, i - self.window_size + 1)
            errors = ae_errors.iloc[start:i+1]

            # Calculate MAE for the current window
            rolling_mae[i] = np.mean(errors)


        #target
        # df['mae'] = rolling_mae


        df['mae'] = ae_errors.rolling(window=self.window_size).mean().fillna(0)



        # decay_factor = 0.90  # Higher weight for recent errors
        # weights = np.array([decay_factor**(self.window_size - i) for i in range(self.window_size)])

        # # Enhanced Weighted MAE Calculation
        # weighted_errors = []
        # for i in range(len(oof_predictions)):
        #     start = max(0, i - self.window_size + 1)
        #     window_errors = ae_errors.iloc[start:i+1]
        #     if len(window_errors) < self.window_size:
        #         adjusted_weights = weights[-len(window_errors):] / np.sum(weights[-len(window_errors):])
        #     else:
        #         adjusted_weights = weights / np.sum(weights)
        #     weighted_errors.append(np.dot(window_errors, adjusted_weights))

        # df['mae'] = pd.Series(weighted_errors, index=df.index)





        #covariates
        for lag in range(1, 3):
            df[f'prediction_lag_{lag}'] = oof_predictions.shift(lag).fillna(0)
            df[f'ae_error_lag_{lag}'] = ae_errors.shift(lag).fillna(0)
            df[f"actual_lag_{lag}"] = real_data.shift(lag).fillna(0)

        for lag in range(1, 5):
            df[f"mae_lag_{lag}"] = df["mae"].shift(lag).fillna(0)

        df['prediction'] = oof_predictions


        #rolling statistics
        df['rolling_mean_ae_3d'] = ae_errors.rolling(3).mean().shift(1).fillna(0)
        df['rolling_std_ae_10d'] = ae_errors.rolling(10).std().shift(1).fillna(0)

        #trend
        df['trend_mae'] = df['mae'].shift(1).fillna(0) - df['mae'].shift(2).fillna(0)
        df['trend_ae'] = ae_errors.shift(1).fillna(0) - ae_errors.shift(2).fillna(0)

        #interaction term
        df['interation_prediction_actual'] = df['prediction'] * df['actual_lag_1']
        df['mae_actual_ratio'] = df['mae'].shift(1).fillna(0) * df['prediction']


        #new metrics
        df['pred_pct_change'] = oof_predictions.iloc[:,0].pct_change().abs().fillna(0)


        df['squared_error'] = np.abs(ae_errors**2).shift(1).fillna(0)

        signed_error = real_data.iloc[:,0] - oof_predictions.iloc[:,0]
        df['overprediction_flag_lag1'] = ((signed_error.shift(1) > 0).astype(int).fillna(0))
















        #exponential moving average




        # df['error'] = ae_errors



        #visualize
        # print(df)

        return df






    def meta_model_preprocessing(self, oof_predictions, real_data, scaler, is_train = False):
        df = self.feature_engineer(oof_predictions, real_data)

        #save df
        if is_train == True:
            df.to_csv("train_df.csv")
        else:
            df.to_csv("test_df.csv")

        if is_train == True:
            self.target_scaler.fit(df[['mae']])

        #scale
        df_scaled = pd.DataFrame(scaler.fit_transform(df), index=df.index, columns=df.columns)

        X , y = df_scaled.drop('mae', axis=1) , df_scaled['mae']

        return X, y





    def get_meta_model(self, meta_model): #get pre trained models

        try:
            if meta_model == "linear booster":

                # best_params = {"n_estimators": 1,  "max_depth": 10,  "min_samples_split" : 10, "min_samples_leaf" : 0.2 }
                # base_estimator = Ridge(alpha = 0.5)

                best_params = {"n_estimators": 1,  "max_depth": 5,  "min_samples_split" : 5, "min_samples_leaf" : 0.2 }
                base_estimator = LinearRegression()

                return LinearBoostRegressor(base_estimator=base_estimator, **best_params)


            elif meta_model == "linear forest":
                best_params = {"n_estimators": 500,  "max_depth": 5,  "min_samples_split" : 5, "min_samples_leaf" : 0.3 , 'max_features' : "log2"}
                base_estimator = LinearRegression()

                return LinearForestRegressor(base_estimator = base_estimator, **best_params)


        except Exception as e:
            raise ValueError(f"Unsupported base model type: {type(meta_model)}")



    def sliding_windows(self, data, seq_length): #helper function to genereate sliding windows
        X, y = [], []

        for i in range(len(data) - seq_length ):
            _x = data.iloc[i:i+seq_length]
            _y = data.iloc[i + seq_length]
            X.append(_x)
            y.append(_y)

        return np.array(X), np.array(y)


    def get_device(self):
        if torch.cuda.is_available():
            device = 'cuda'
        else:
            device = 'cpu'

        return device


    @staticmethod
    def to_device(data, device):
        if isinstance(data, (list, tuple)):
            return [to_device(x, device) for x in data]
        elif isinstance(data, torch.Tensor):  # Only move tensors to the device
            return data.to(device, non_blocking=True)
        else:
            return data  # For non-tensor types (e.g., strings), return as is


    @torch.no_grad()
    def inference(self, model, data_loader):
        model.eval()  # Set the model to evaluation mode
        predictions = []
        actuals = []

        for batch in data_loader:
            input, output = batch  # Unpack your batch into input and output
            outputs = model(input) # Perform the forward pass

            # Move outputs and actuals back to CPU and append to lists
            predictions.append(outputs.cpu())
            actuals.append(output.cpu())

        # Concatenate all predictions and actuals into single tensors
        predictions = torch.cat(predictions, dim=0)
        actuals = torch.cat(actuals, dim=0)

        return predictions, actuals


    def data_preprocessing_deep_learning (self, train_data, test_data, scaler, time_window):
        #scale data
        train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data))
        test_data_scaled = pd.DataFrame(scaler.transform(test_data))

        #create sliding windows
        X_train, y_train = self.sliding_windows(train_data_scaled, time_window)
        X_test, y_test = self.sliding_windows(test_data_scaled, time_window)

        #move data to tensors
        train_df = Custom_df(X_train, y_train)
        test_df = Custom_df(X_test, y_test)

        #create dataloaders (to perform training/inference in batch)
        batch_size = 128
        num_workers = 2
        pin_memory = True if self.device == "cuda" else False
        train_loader = DataLoader(train_df, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory, persistent_workers=True)
        test_loader = DataLoader(test_df, batch_size=batch_size*2, shuffle=False, num_workers=num_workers, pin_memory=pin_memory, persistent_workers=True)

        #move to device
        train_loader = DeviceDataLoader(train_loader, self.device)
        test_loader = DeviceDataLoader(test_loader, self.device)

        return train_loader, test_loader


    def data_preprocessing_machine_learning(self, train_data, test_data, scaler, time_window):
        #convert data to numpy arrays
        if isinstance(train_data, torch.Tensor):
            train_data = train_data.numpy()
        if isinstance(test_data, torch.Tensor):
            test_data = test_data.numpy()

        #scale data

        if isinstance(train_data, pd.Series):
            train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data.values.reshape(-1,1)))
            test_data_scaled = pd.DataFrame(scaler.transform(test_data.values.reshape(-1,1)))

        else:
            train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data))
            test_data_scaled = pd.DataFrame(scaler.transform(test_data))

        #create sliding windows
        X_train, y_train = self.sliding_windows(train_data_scaled, time_window)
        X_test, y_test = self.sliding_windows(test_data_scaled, time_window)

        # Flatten the sliding windows
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)

        return X_train, y_train, X_test, y_test


    def k_fold_model_prediction(self, model):
        try:
            if isinstance(model, LinearBoostRegressor): #Linear Boost model predictions

                #compute k fold out of sample predictions
                oof_predictions = np.zeros(len(self.train_data))

                with self.lock:
                    for fold, (train_idx, test_idx) in enumerate(self.tscv.split(range(len(self.train_data)))):

                        #save first fold (unused)
                        if fold == 0:
                            self.first_fold_idx = test_idx[self.window_size]


                        #data preprocessing
                        train_df, test_df = self.train_data.iloc[train_idx], self.train_data.iloc[test_idx]

                        #X,y
                        scaler = RobustScaler()
                        X_train, y_train, X_test, y_test = self.data_preprocessing_machine_learning(train_df, test_df, scaler, self.window_size)

                        #predict
                        test_prediction = model.predict(X_test)

                        #inverse scale
                        test_prediction = scaler.inverse_transform(test_prediction.reshape(-1, 1))

                        # Store predictions in the correct indices
                        oof_predictions[test_idx[self.window_size:]] = test_prediction.flatten()


                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.train_data[self.window_size:].index, y = self.train_data[self.window_size:].squeeze(), label="Train Data")
                    plt.title('Out of sample XGBoost Predictions on Train set')
                    sns.lineplot(x= self.train_data[self.first_fold_idx:].index, y=oof_predictions[self.first_fold_idx:].squeeze(), label="Out of sample XGBoost predictions on Train set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    r2 = r2_score(self.train_data[self.first_fold_idx:].values, oof_predictions[self.first_fold_idx:])
                    print(f"R2 score: {r2}")

                # return (oof_predictions[self.first_fold_idx:], train_volatility)
                return (oof_predictions[self.first_fold_idx:])


            elif isinstance(model, torch.nn.Module):


                #compute k fold out of sample predictions
                oof_predictions = np.zeros(len(self.train_data))

                with self.lock:
                    for fold, (train_idx, test_idx) in enumerate(self.tscv.split(range(len(self.train_data)))):

                        #save first fold (unused)
                        if fold == 0:
                            self.first_fold_idx = test_idx[self.window_size]

                        #data preprocessing
                        train_df, test_df = self.train_data.iloc[train_idx], self.train_data.iloc[test_idx]

                        #X,y
                        scaler = MinMaxScaler(feature_range=(-1, 1))
                        train_loader, test_loader = self.data_preprocessing_deep_learning(train_df, test_df, scaler, self.window_size)

                        #predict
                        test_prediction, test_actual = self.inference(model, test_loader)

                        #inverse scale
                        test_prediction = scaler.inverse_transform(test_prediction)

                        # Store predictions in the correct indices
                        oof_predictions[test_idx[self.window_size:]] = test_prediction.flatten()



                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.train_data[self.window_size:].index, y = self.train_data[self.window_size:].squeeze(), label="Train Data")
                    sns.lineplot(x= self.train_data[self.first_fold_idx:].index, y=oof_predictions[self.first_fold_idx:].squeeze(), label=f"Out of sample {model.__class__.__name__} predictions on Train set")
                    plt.title(f"Out of sample {model.__class__.__name__} predictions on Train set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    r2 = r2_score(self.train_data[self.first_fold_idx:].values, oof_predictions[self.first_fold_idx:])
                    print(f"R2 score: {r2}")

                    # Explicit cleanup
                    del train_loader, test_loader
                    torch.cuda.empty_cache()

                # return (oof_predictions[self.first_fold_idx:], train_volatility)
                return (oof_predictions[self.first_fold_idx:])

            else:
                raise ValueError(f"Unsupported base model type: {type(model)}")


        except Exception as e:
            print(f"Error processing model {model}: {e}")
            raise e


    def model_prediction(self, model, oof_predictions):
        try:
            if isinstance(model, LinearBoostRegressor): #Linear Boost model predictions


                #data preprocessing
                with self.lock:
                    scaler = RobustScaler()
                    X_train, y_train = self.meta_model_preprocessing(oof_predictions, self.train_data[self.first_fold_idx:], scaler, is_train = True)

                    #train meta model
                    self.meta_model.fit(X_train, y_train)
                    print('meta model trained')
                    MAE_predictions = self.meta_model.predict(X_train)

                    #unscale
                    MAE_predictions = self.target_scaler.inverse_transform(MAE_predictions.reshape(-1, 1))
                    y_train = self.target_scaler.inverse_transform(y_train.to_numpy().reshape(-1, 1))



                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.train_data[self.first_fold_idx :].index, y = y_train.ravel(), label="train MAE")
                    sns.lineplot(x= self.train_data[self.first_fold_idx :].index, y=MAE_predictions.squeeze(), label="MAE predicted")
                    plt.title('MAE predictions on Train set Linear Boost')
                    plt.xlabel('Date')
                    plt.ylabel('MAE')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    #compute r2
                    r2 = r2_score(y_train, MAE_predictions)
                    print(f"R2 score: {r2}")

                    #test predictions
                    test_prediction = model.predict(self.X_test)

                    #unscale
                    test_prediction = self.Robustscaler.inverse_transform(test_prediction.reshape(-1, 1))

                    #data preprocessing
                    X_test, y_test = self.meta_model_preprocessing(test_prediction, self.test_data[self.window_size:], scaler)

                    #meta model predict
                    MAE_predictions = self.meta_model.predict(X_test)

                    #unscale predicions
                    MAE_predictions = self.target_scaler.inverse_transform(MAE_predictions.reshape(-1, 1))
                    y_test = self.target_scaler.inverse_transform(y_test.to_numpy().reshape(-1, 1))

                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.test_data[self.window_size :].index, y = y_test.ravel(), label="Test MAE")
                    sns.lineplot(x= self.test_data[self.window_size:].index, y=MAE_predictions.squeeze(), label="MAE predicted")
                    plt.title('MAE predictions on Test set Linear Boost')
                    plt.xlabel('Date')
                    plt.ylabel('MAE')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    #compute r2
                    r2 = r2_score(y_test, MAE_predictions)
                    print(f"R2 score: {r2}")


                return (torch.tensor(test_prediction, dtype=torch.float32, device = self.device).detach().squeeze() , torch.tensor(MAE_predictions, dtype=torch.float32).detach())
                # return (torch.tensor(test_prediction, dtype=torch.float32, device = self.device).detach().squeeze() , torch.tensor(y_test, dtype=torch.float32).detach())

            elif isinstance(model, torch.nn.Module):

                #data preprocessing
                scaler = RobustScaler()
                with self.lock:
                    X_train, y_train = self.meta_model_preprocessing(oof_predictions, self.train_data[self.first_fold_idx:], scaler, is_train = True)


                    #train meta model
                    self.meta_model.fit(X_train, y_train)
                    print('meta model trained')
                    MAE_predictions = self.meta_model.predict(X_train)

                    #unscale
                    MAE_predictions = self.target_scaler.inverse_transform(MAE_predictions.reshape(-1, 1))
                    y_train = self.target_scaler.inverse_transform(y_train.to_numpy().reshape(-1, 1))


                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.train_data[self.first_fold_idx :].index, y = y_train.ravel(), label="train MAE")
                    sns.lineplot(x= self.train_data[self.first_fold_idx :].index, y=MAE_predictions.squeeze(), label="MAE predicted")
                    plt.title(f'MAE predictions on Train set {model.__class__.__name__}')
                    plt.xlabel('Date')
                    plt.ylabel('MAE')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    #compute r2
                    r2 = r2_score(y_train, MAE_predictions)
                    print(f"R2 score: {r2}")


                    #test predictions
                    test_prediction, test_actual = self.inference(model, self.test_loader)

                    #unscale
                    test_prediction = self.MinMaxscaler.inverse_transform(test_prediction.reshape(-1, 1))


                    #data preprocessing
                    X_test, y_test = self.meta_model_preprocessing(test_prediction, self.test_data[self.window_size:], scaler)


                    #meta model predict
                    MAE_predictions = self.meta_model.predict(X_test)

                    #unscale predicions
                    MAE_predictions = self.target_scaler.inverse_transform(MAE_predictions.reshape(-1, 1))
                    y_test = self.target_scaler.inverse_transform(y_test.to_numpy().reshape(-1, 1))

                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.test_data[self.window_size :].index, y = y_test.ravel(), label="Test MAE")
                    sns.lineplot(x= self.test_data[self.window_size:].index, y=MAE_predictions.squeeze(), label="MAE predicted")
                    plt.title(f'MAE predictions on Test set {model.__class__.__name__}')
                    plt.xlabel('Date')
                    plt.ylabel('MAE')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    #compute r2
                    r2 = r2_score(y_test, MAE_predictions)
                    print(f"R2 score: {r2}")




                return (torch.tensor(test_prediction, dtype=torch.float32, device=self.device).detach().squeeze() , torch.tensor(MAE_predictions, dtype=torch.float32).detach())
                # return (torch.tensor(test_prediction, dtype=torch.float32, device=self.device).detach().squeeze() , torch.tensor(y_test, dtype=torch.float32).detach())

            else:
                raise ValueError(f"Unsupported base model type: {type(model)}")




        except Exception as e:
            print(f"Error processing model {model}: {e}")
            raise e


    def predict(self):
        #k fold train predictions
        print("out of sample predictions")
        with ThreadPoolExecutor() as executor: #it creates a pool of worker threads (the with statement ensures that the pool of threads is cleaned up automatically after the execution)
            results = list(executor.map(self.k_fold_model_prediction, self.trained_models)) #the task each thread will execute


        # Combine oof predictions into a list
        self.oof_predictions = [item for item in results]


        #test predictions
        with ThreadPoolExecutor() as executor: #it creates a pool of worker threads (the with statement ensures that the pool of threads is cleaned up automatically after the execution)
            results = list(executor.map(self.model_prediction, self.trained_models, self.oof_predictions)) #the task each thread will execute


        # Combine predictions into a single tensor
        self.test_predictions = torch.stack([item[0] for item in results], dim=1).squeeze()
        self.test_error_predictions = torch.stack([item[1] for item in results], dim=1).squeeze()

        print("test predictions")
        print(self.test_predictions)



        print("test error predictions")
        print(self.test_error_predictions)


        #filter the 3 best errors
        best_kerrors, indices = torch.topk(self.test_error_predictions, k=self.k, largest=False, sorted=False)
        print("best k errors")
        print(best_kerrors)


        #normalize the errors
        std = best_kerrors.std(dim=1, keepdim=True) + 1e-8 #for numerical stability (to avoid division by zero)
        print("std")
        print(std[:10])
        mean = best_kerrors.mean(dim=1, keepdim=True)
        print("mean")
        print(mean[:10])
        normalized_errors = (self.test_error_predictions - mean) / std
        print("unmasked normalized errors")
        print(normalized_errors)


        #apply mask
        mask = torch.zeros_like(self.test_error_predictions, dtype=torch.bool)
        mask.scatter_(dim=1, index=indices, value=True)
        normalized_errors[~mask] = float('inf')
        print("normalized errors")
        print(normalized_errors)


        #apply softmax
        weights_df = F.softmax(-normalized_errors, dim=1)
        print("weights")
        print(weights_df)

        weights_df = F.softmax(-normalized_errors * self.temperature, dim=1)
        print("temperature weights")
        print(weights_df)

        #combine predictions
        self.ensemble_preds = self.test_predictions * weights_df
        self.ensemble_preds = self.ensemble_preds.sum(dim=1)


        return self.ensemble_preds

Linear booster meta model

In [ ]:
#using the class
train_data = {}
test_data = {}

trained_models = [Linear_booster_model, CNN_model, LSTM_model, LSTM_CNN_model, parallelCNNLSTM_model]

for name , _ in datasets.items():

    # if name == 'sp500':
    #     continue

    # if name == 'eur_usd':
    #     continue

    print(f"ensembling predictions on {name} dataset:")
    meta_model = "linear booster"
    # meta_model = "linear forest"

    train_data[name] = val_residuals[name]
    train_data[name].index = pd.to_datetime(val_residuals[name].index)

    test_data[name] = test_residuals[name]
    test_data[name].index = pd.to_datetime(test_residuals[name].index)
    window_size = 10
    error_window = 10

    if name == 'eur_usd':
        # k=4
        # temperature = 1 #to control weight sharpness

        k=3
        temperature = 0.65 #to control weight sharpness
    elif name == 'sp500':
        temperature = 0.05
        k=3 #number of models to make the ensemble at each time step



    ade = ADE([trained_model[name] for trained_model in trained_models], meta_model, train_data[name], test_data[name], window_size, error_window, k, temperature) #k represent the number of model to select from the batch of models
    final_preds = ade.predict()


    #visualize
    ensemble_preds = pd.DataFrame(final_preds.detach().numpy(), index = test_data[name][window_size:].index)
    # ensemble_preds.index = pd.to_datetime(test_data[name][window_size*2:].index)

    plt.figure(figsize=(12, 6))
    plt.title(f"ADE ensemble predictions {name}")
    sns.lineplot(x = ensemble_preds.index, y = ensemble_preds.squeeze(), label = "ADE", color = "red")
    sns.lineplot(x = test_data[name][window_size*2:].index, y = test_data[name][window_size*2:].squeeze(), label = "test set", color = "blue")
    # sns.lineplot(x = test_data[name][window_size:].index, y = test_data[name][window_size:].squeeze(), label = "test set", color = "blue")
    plt.ylabel('Close price residuals')
    plt.xlabel('Date')
    plt.xticks(rotation=45)  # Rotate x-axis labels
    plt.tight_layout() # Automatically adjusts the layout
    plt.legend()
    plt.show()


    #compute metrics
    # mse = mean_squared_error(test_data[name][window_size*2:], ensemble_preds)
    # rmse = np.sqrt(mse)
    # mae = mean_absolute_error(test_data[name][window_size*2:], ensemble_preds)
    # r2 = r2_score(test_data[name][window_size*2:], ensemble_preds)
    # mape = mean_absolute_percentage_error(test_data[name][window_size*2:], ensemble_preds)
    # smape_value = smape(test_data[name][window_size*2:], ensemble_preds)

    mse = mean_squared_error(test_data[name][window_size:], ensemble_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_data[name][window_size:], ensemble_preds)
    r2 = r2_score(test_data[name][window_size:], ensemble_preds)
    mape = mean_absolute_percentage_error(test_data[name][window_size:], ensemble_preds)
    smape_value = smape(test_data[name][window_size:], ensemble_preds)

    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"r2:  {r2}")
    print(f"MAPE: {mape}")
    print(f"SMAPE: {smape_value}")


    #combine ensemble predictions with ARIMA model
    final_preds = SARIMA_test_predictions[name][window_size:] + ensemble_preds.iloc[:,0]

    #visualize
    plt.figure(figsize=(12, 6))
    plt.title(f"SARIMA + ensemble predictions {name}")
    sns.lineplot(x = final_preds.index, y = final_preds.squeeze(), label = "SARIMA + ensemble predictions", color = "red")
    sns.lineplot(x = Close_price[name].loc[final_preds.index].index, y = Close_price[name].loc[final_preds.index].squeeze(), label = "test set", color = "blue")
    plt.ylabel('Close price')
    plt.xlabel('Date')
    plt.xticks(rotation=45)  # Rotate x-axis labels
    plt.tight_layout() # Automatically adjusts the layout
    plt.legend()
    plt.show()


    #compute metrics
    mse = mean_squared_error(Close_price[name].loc[final_preds.index], final_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(Close_price[name].loc[final_preds.index], final_preds)
    r2 = r2_score(Close_price[name].loc[final_preds.index], final_preds)
    mape = mean_absolute_percentage_error(Close_price[name].loc[final_preds.index], final_preds)
    smape_value = smape(Close_price[name].loc[final_preds.index], final_preds)

    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"r2:  {r2}")
    print(f"MAPE: {mape}")
    print(f"SMAPE: {smape_value}")

Output hidden; open in https://colab.research.google.com to view.

#Static ensemble techniques

Stacking

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from google.colab import drive
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from joblib import Parallel, delayed, dump, load
import xgboost as xgb
import time
import yfinance as yf
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor
import threading
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import TimeSeriesSplit

#suppress warnings
import warnings
warnings.filterwarnings('ignore')


class OverlappingTimeSeriesSplit():
    def __init__(self, n_splits=5, window_size=10):
        self.n_splits = n_splits
        self.window_size = window_size

    def split(self, X):
        n_samples = len(X)
        fold_size = n_samples // (self.n_splits + 1)  # Ensure proper splitting

        for i in range(1, self.n_splits + 1):  # Start from 1 to avoid empty train set
            train_end = i * fold_size
            train_idx = range(train_end)  # Train set includes all data before validation

            val_start = max(0, train_end - self.window_size)
            val_end = train_end + fold_size
            val_idx = range(val_start, min(val_end, n_samples))  # Ensure within bounds

            yield list(train_idx), list(val_idx)

def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    smape_value = np.mean(numerator / denominator)  # Renamed variable to avoid conflict
    return smape_value

#deep learning meta model architecture
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size=512, num_layers=3, dropout=0.3):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes

        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, self.num_classes)

        # Activation and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)

        # Pass only the last output
        lstm_out = lstm_out[:, -1, :]

        # Fully connected layers with ReLU and Dropout
        x = self.relu(self.fc1(lstm_out))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)  # Final prediction
        return x


#deep learning meta model architecture
class ParallelCNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(out_features=128), #linear layer that automatically infer the input size
            nn.ReLU()
        )
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc_lstm = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128*2, num_classes)

    def forward(self, x):
        #cnn takes input of shape (batch_size, channels, seq_len)
        x_cnn = x.permute(0, 2, 1)
        out_cnn = self.cnn(x_cnn)
        # lstm takes input of shape (batch_size, seq_len, input_size)
        out_lstm, _ = self.lstm(x)
        out_lstm = self.fc_lstm(out_lstm[:, -1, :])
        out = torch.cat([out_cnn, out_lstm], dim=1)
        out = self.fc(out)
        return out


class Custom_df(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index): #this method allows to retrieve a specific sample from the dataset based on its index (the index is passed to this method)
        input = torch.tensor(np.array(self.x[index]), dtype=torch.float32) #convert np array into a tensor
        target = torch.tensor(np.array(self.y[index]), dtype=torch.float32)
        return input, target

    def __len__(self):
        return len(self.x)


class DeviceDataLoader (): #receive a dataloader and move to the correct device
  def __init__(self, dl : DataLoader, device):
    self.dl = dl
    self.device = device

  def __iter__(self):
    for batch in iter(self.dl): #iter will reset the iterator every time the __iter__ mathod gets called
      yield Stacking.to_device(batch, device)

  def __len__(self):
    return len(self.dl)



class Stacking ():
    def __init__(self, trained_models: list, meta_model, train_data: pd.DataFrame, test_data: pd.DataFrame, window_size : int = 10):
        self.trained_models = trained_models
        self.meta_model = self.get_meta_model(meta_model) #accept a string as an input
        self.train_data = pd.DataFrame(train_data)
        self.test_data = pd.DataFrame(test_data)
        self.MinMaxscaler = MinMaxScaler(feature_range=(-1, 1))
        self.Robustscaler = RobustScaler()
        # self.error_scaler = MinMaxScaler(feature_range=(-1, 1))
        self.window_size = window_size
        self.oof_predictions = []
        self.test_error_predictions = []
        self.device = self.get_device()
        self.lock = threading.Lock()
        self.ensemble_preds = None
        self.tscv = OverlappingTimeSeriesSplit(n_splits=5, window_size=self.window_size) #TimeSeriesSplit object to generate time ware time splits
        self.fist_fold_idx = None
        self.test_predictions = None


        #if there are deep learning models
        if any(isinstance(model, torch.nn.Module) for model in trained_models):
            self.train_loader, self.test_loader = self.data_preprocessing_deep_learning(self.train_data, self.test_data, self.MinMaxscaler, self.window_size)

        #if there are machine learning models
        if any(not isinstance(model, torch.nn.Module) for model in trained_models):
            self.X_train, self.y_train, self.X_test, self.y_test = self.data_preprocessing_machine_learning(self.train_data, self.test_data, self.Robustscaler, self.window_size)



    def feature_engineer(self, volatility, prediction):
        #target
        volatility['error_volatility'] = volatility


        #covariates
        # dataset['prediction'] = pd.DataFrame(prediction)
        # dataset['predicted volatility'] = dataset['prediction'].ewm(span=10, adjust=False).std().fillna(0)

        # for lag in range(1, 5):
        for lag in range(1, 3):
            volatility[f'lag_{lag}'] = volatility['error_volatility'].shift(lag).fillna(0)



    def get_meta_model(self, meta_model):
        try:
            if meta_model == "linear_regression":
                from sklearn.linear_model import LinearRegression
                return LinearRegression()

            elif meta_model == "ridge":
                from sklearn.linear_model import Ridge
                return Ridge()

            elif meta_model == "gradient_boosting":
                from sklearn.ensemble import GradientBoostingRegressor
                return GradientBoostingRegressor()

            elif meta_model == "random_forest":
                from sklearn.ensemble import RandomForestRegressor
                return RandomForestRegressor()

            else:
                raise ValueError(f"Unsupported meta model: {meta_model}")

        except Exception as e:
            print(f"Meta model error: {e}")
            raise e



    def sliding_windows(self, data, seq_length): #helper function to genereate sliding windows
        X, y = [], []

        for i in range(len(data) - seq_length ):
            _x = data.iloc[i:i+seq_length]
            _y = data.iloc[i + seq_length]
            X.append(_x)
            y.append(_y)

        return np.array(X), np.array(y)


    def get_device(self):
        if torch.cuda.is_available():
            device = 'cuda'
        else:
            device = 'cpu'

        return device


    @staticmethod
    def to_device(data, device):
        if isinstance(data, (list, tuple)):
            return [to_device(x, device) for x in data]
        elif isinstance(data, torch.Tensor):  # Only move tensors to the device
            return data.to(device, non_blocking=True)
        else:
            return data  # For non-tensor types (e.g., strings), return as is


    @torch.no_grad()
    def inference(self, model, data_loader):
        model.eval()  # Set the model to evaluation mode
        predictions = []
        actuals = []

        for batch in data_loader:
            input, output = batch  # Unpack your batch into input and output
            outputs = model(input) # Perform the forward pass

            # Move outputs and actuals back to CPU and append to lists
            predictions.append(outputs.cpu())
            actuals.append(output.cpu())

        # Concatenate all predictions and actuals into single tensors
        predictions = torch.cat(predictions, dim=0)
        actuals = torch.cat(actuals, dim=0)

        return predictions, actuals


    def data_preprocessing_deep_learning (self, train_data, test_data, scaler, time_window):
        #scale data
        train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data))
        test_data_scaled = pd.DataFrame(scaler.transform(test_data))

        #create sliding windows
        X_train, y_train = self.sliding_windows(train_data_scaled, time_window)
        X_test, y_test = self.sliding_windows(test_data_scaled, time_window)

        #move data to tensors
        train_df = Custom_df(X_train, y_train)
        test_df = Custom_df(X_test, y_test)

        #create dataloaders (to perform training/inference in batch)
        batch_size = 128
        num_workers = 2
        pin_memory = True if self.device == "cuda" else False
        train_loader = DataLoader(train_df, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory, persistent_workers=True)
        test_loader = DataLoader(test_df, batch_size=batch_size*2, shuffle=False, num_workers=num_workers, pin_memory=pin_memory, persistent_workers=True)

        #move to device
        train_loader = DeviceDataLoader(train_loader, self.device)
        test_loader = DeviceDataLoader(test_loader, self.device)

        return train_loader, test_loader


    def data_preprocessing_machine_learning(self, train_data, test_data, scaler, time_window):
        #convert data to numpy arrays
        if isinstance(train_data, torch.Tensor):
            train_data = train_data.numpy()
        if isinstance(test_data, torch.Tensor):
            test_data = test_data.numpy()

        #scale data

        if isinstance(train_data, pd.Series):
            train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data.values.reshape(-1,1)))
            test_data_scaled = pd.DataFrame(scaler.transform(test_data.values.reshape(-1,1)))

        else:
            train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data))
            test_data_scaled = pd.DataFrame(scaler.transform(test_data))

        #create sliding windows
        X_train, y_train = self.sliding_windows(train_data_scaled, time_window)
        X_test, y_test = self.sliding_windows(test_data_scaled, time_window)

        # Flatten the sliding windows
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)

        return X_train, y_train, X_test, y_test




    def k_fold_model_prediction(self, model):
        try:
            if isinstance(model, LinearBoostRegressor): #Linear Boost model predictions

                #compute k fold out of sample predictions
                oof_predictions = np.zeros(len(self.train_data))

                with self.lock:
                    for fold, (train_idx, test_idx) in enumerate(self.tscv.split(range(len(self.train_data)))):

                        #save first fold (unused)
                        if fold == 0:
                            self.fist_fold_idx = test_idx[self.window_size]


                        #data preprocessing
                        train_df, test_df = self.train_data.iloc[train_idx], self.train_data.iloc[test_idx]

                        #X,y
                        scaler = RobustScaler()
                        X_train, y_train, X_test, y_test = self.data_preprocessing_machine_learning(train_df, test_df, scaler, self.window_size)

                        #predict
                        test_prediction = model.predict(X_test)

                        #inverse scale
                        test_prediction = scaler.inverse_transform(test_prediction.reshape(-1, 1))

                        # Store predictions in the correct indices
                        oof_predictions[test_idx[self.window_size:]] = test_prediction.flatten()


                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.train_data[self.window_size:].index, y = self.train_data[self.window_size:].squeeze(), label="Train Data")
                    plt.title('Out of sample XGBoost Predictions on Train set')
                    sns.lineplot(x= self.train_data[self.fist_fold_idx:].index, y=oof_predictions[self.fist_fold_idx:].squeeze(), label="Out of sample XGBoost predictions on Train set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    # print(self.train_data[fist_fold_idx:].shape)
                    # print(oof_predictions.shape)

                    r2 = r2_score(self.train_data[self.fist_fold_idx:].values, oof_predictions[self.fist_fold_idx:])
                    print(f"R2 score: {r2}")


                return (oof_predictions[self.fist_fold_idx:])


            elif isinstance(model, torch.nn.Module):


                #compute k fold out of sample predictions
                oof_predictions = np.zeros(len(self.train_data))

                with self.lock:
                    for fold, (train_idx, test_idx) in enumerate(self.tscv.split(range(len(self.train_data)))):

                        #save first fold (unused)
                        if fold == 0:
                            self.fist_fold_idx = test_idx[self.window_size]

                        #data preprocessing
                        train_df, test_df = self.train_data.iloc[train_idx], self.train_data.iloc[test_idx]

                        #X,y
                        scaler = MinMaxScaler(feature_range=(-1, 1))
                        train_loader, test_loader = self.data_preprocessing_deep_learning(train_df, test_df, scaler, self.window_size)

                        #predict
                        test_prediction, test_actual = self.inference(model, test_loader)

                        #inverse scale
                        test_prediction = scaler.inverse_transform(test_prediction)

                        # Store predictions in the correct indices
                        oof_predictions[test_idx[self.window_size:]] = test_prediction.flatten()



                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.train_data[self.window_size:].index, y = self.train_data[self.window_size:].squeeze(), label="Train Data")
                    sns.lineplot(x= self.train_data[self.fist_fold_idx:].index, y=oof_predictions[self.fist_fold_idx:].squeeze(), label=f"Out of sample {model.__class__.__name__} predictions on Train set")
                    plt.title(f"Out of sample {model.__class__.__name__} predictions on Train set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    r2 = r2_score(self.train_data[self.fist_fold_idx:].values, oof_predictions[self.fist_fold_idx:])
                    print(f"R2 score: {r2}")


                    # Explicit cleanup
                    del train_loader, test_loader
                    torch.cuda.empty_cache()



                return (oof_predictions[self.fist_fold_idx:])


            else:
                raise ValueError(f"Unsupported base model type: {type(model)}")


        except Exception as e:
            print(f"Error processing model {model}: {e}")
            raise e




    def model_prediction(self, model):
        try:
            if isinstance(model, LinearBoostRegressor): #Linear Boost model predictions

                with self.lock:
                    test_prediction = model.predict(self.X_test)

                    #inverse scale
                    test_prediction = self.Robustscaler.inverse_transform(test_prediction.reshape(-1, 1))

                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.test_data[self.window_size:].index, y = self.test_data[self.window_size:].squeeze(), label="Train Data")
                    plt.title('XGBoost Predictions on Test set')
                    sns.lineplot(x= self.test_data[self.window_size:].index, y=test_prediction.squeeze(), label="Out of sample XGBoost predictions on Train set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()


                    r2 = r2_score(self.test_data[self.window_size:].values, test_prediction)
                    print(f"R2 score: {r2}")


                return (test_prediction)


            elif isinstance(model, torch.nn.Module):
                #train predictions
                with self.lock: #the lock prevents multiple threads from accessing shared data at the same time
                    test_prediction, test_actual = self.inference(model, self.test_loader)

                    # Inverse Scale
                    test_prediction = self.MinMaxscaler.inverse_transform(test_prediction)

                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.test_data[self.window_size:].index, y = self.test_data[self.window_size:].squeeze(), label="Train Data")
                    sns.lineplot(x= self.test_data[self.window_size:].index, y=test_prediction.squeeze(), label=f"{model.__class__.__name__} predictions on Test set")
                    plt.title(f"{model.__class__.__name__} predictions on Test set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    r2 = r2_score(self.test_data[self.window_size:].values, test_prediction)
                    print(f"R2 score: {r2}")

                return (test_prediction)


            else:
                raise ValueError(f"Unsupported base model type: {type(model)}")


        except Exception as e:
            print(f"Error processing model {model}: {e}")
            raise e


    def predict(self):
        #parallel execution
        print("Out of fold predictions on train set")
        with ThreadPoolExecutor() as executor: #it creates a pool of worker threads (the with statement ensures that the pool of threads is cleaned up automatically after the execution)
            results = list(executor.map(self.k_fold_model_prediction, self.trained_models)) #the task each thread will execute

        # Combine oof predictions into a single array (meta features)
        self.oof_predictions = np.hstack([item.reshape(-1, 1) for item in results])
        # print(self.oof_predictions.shape)


        #train meta model
        self.meta_model.fit(self.oof_predictions, self.Robustscaler.inverse_transform(self.y_train[(self.fist_fold_idx -self.window_size):]))
        print("meta model trained")


        #retrain model on all train set
        print("Prediction on test set")
        with ThreadPoolExecutor() as executor: #it creates a pool of worker threads (the with statement ensures that the pool of threads is cleaned up automatically after the execution)
            test_results = list(executor.map(self.model_prediction, self.trained_models)) #the task each thread will execute

        #stack test predictions
        self.test_predictions = np.hstack([item.reshape(-1, 1) for item in test_results])
        # print(self.test_predictions.shape)

        #use meta model to generate final predictions
        final_preds = self.meta_model.predict(self.test_predictions)

        return final_preds

In [ ]:
#using the class
train_data = {}
test_data = {}

trained_models = [Linear_booster_model, CNN_model, LSTM_model, LSTM_CNN_model, parallelCNNLSTM_model]

for name , _ in datasets.items():

    # if name == 'sp500':
    #     continue

    # if name == 'eur_usd':
    #     continue

    print(f"ensembling predictions on {name} dataset:")

    meta_model = "linear_regression"

    train_data[name] = val_residuals[name]
    train_data[name].index = pd.to_datetime(val_residuals[name].index)

    test_data[name] = test_residuals[name]
    test_data[name].index = pd.to_datetime(test_residuals[name].index)
    window_size = 10


    #stacking
    stack = Stacking([trained_model[name] for trained_model in trained_models], meta_model, train_data[name], test_data[name], window_size) #k represent the number of model to select from the batch of models
    final_preds = stack.predict()


    #visualize
    ensemble_preds = pd.DataFrame(final_preds, index = test_data[name][window_size:].index)

    plt.figure(figsize=(12, 6))
    plt.title(f"Stacking ensemble predictions {name}")
    sns.lineplot(x = ensemble_preds.index, y = ensemble_preds.squeeze(), label = "ADE", color = "red")
    # sns.lineplot(x = test_data[name][window_size*2:].index, y = test_data[name][window_size*2:].squeeze(), label = "test set", color = "blue")
    sns.lineplot(x = test_data[name][window_size:].index, y = test_data[name][window_size:].squeeze(), label = "test set", color = "blue")
    plt.ylabel('Close price residuals')
    plt.xlabel('Date')
    plt.xticks(rotation=45)  # Rotate x-axis labels
    plt.tight_layout() # Automatically adjusts the layout
    plt.legend()
    plt.show()


    #compute metrics
    # mse = mean_squared_error(test_data[name][window_size*2:], ensemble_preds)
    # rmse = np.sqrt(mse)
    # mae = mean_absolute_error(test_data[name][window_size*2:], ensemble_preds)
    # r2 = r2_score(test_data[name][window_size*2:], ensemble_preds)
    # mape = mean_absolute_percentage_error(test_data[name][window_size*2:], ensemble_preds)
    # smape_value = smape(test_data[name][window_size*2:], ensemble_preds)

    mse = mean_squared_error(test_data[name][window_size:], ensemble_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_data[name][window_size:], ensemble_preds)
    r2 = r2_score(test_data[name][window_size:], ensemble_preds)
    mape = mean_absolute_percentage_error(test_data[name][window_size:], ensemble_preds)
    smape_value = smape(test_data[name][window_size:], ensemble_preds)

    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"r2:  {r2}")
    print(f"MAPE: {mape}")
    print(f"SMAPE: {smape_value}")


    #combine ensemble predictions with ARIMA model
    final_preds = SARIMA_test_predictions[name][window_size:] + ensemble_preds.iloc[:,0]

    #visualize
    plt.figure(figsize=(12, 6))
    plt.title(f"SARIMA + ensemble predictions {name}")
    sns.lineplot(x = final_preds.index, y = final_preds.squeeze(), label = "SARIMA + ensemble predictions", color = "red")
    sns.lineplot(x = Close_price[name].loc[final_preds.index].index, y = Close_price[name].loc[final_preds.index].squeeze(), label = "test set", color = "blue")
    plt.ylabel('Close price')
    plt.xlabel('Date')
    plt.xticks(rotation=45)  # Rotate x-axis labels
    plt.tight_layout() # Automatically adjusts the layout
    plt.legend()
    plt.show()


    #compute metrics
    mse = mean_squared_error(Close_price[name].loc[final_preds.index], final_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(Close_price[name].loc[final_preds.index], final_preds)
    r2 = r2_score(Close_price[name].loc[final_preds.index], final_preds)
    mape = mean_absolute_percentage_error(Close_price[name].loc[final_preds.index], final_preds)
    smape_value = smape(Close_price[name].loc[final_preds.index], final_preds)

    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"r2:  {r2}")
    print(f"MAPE: {mape}")
    print(f"SMAPE: {smape_value}")

Output hidden; open in https://colab.research.google.com to view.

Static Weighted Average

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from google.colab import drive
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from joblib import Parallel, delayed, dump, load
import xgboost as xgb
import time
import yfinance as yf
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor
import threading
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import TimeSeriesSplit

#suppress warnings
import warnings
warnings.filterwarnings('ignore')


class OverlappingTimeSeriesSplit():
    def __init__(self, n_splits=5, window_size=10):
        self.n_splits = n_splits
        self.window_size = window_size

    def split(self, X):
        n_samples = len(X)
        fold_size = n_samples // (self.n_splits + 1)  # Ensure proper splitting

        for i in range(1, self.n_splits + 1):  # Start from 1 to avoid empty train set
            train_end = i * fold_size
            train_idx = range(train_end)  # Train set includes all data before validation

            val_start = max(0, train_end - self.window_size)
            val_end = train_end + fold_size
            val_idx = range(val_start, min(val_end, n_samples))  # Ensure within bounds

            yield list(train_idx), list(val_idx)

def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    smape_value = np.mean(numerator / denominator)  # Renamed variable to avoid conflict
    return smape_value

#deep learning meta model architecture
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size=512, num_layers=3, dropout=0.3):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes

        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, self.num_classes)

        # Activation and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # LSTM output
        lstm_out, _ = self.lstm(x)

        # Pass only the last output
        lstm_out = lstm_out[:, -1, :]

        # Fully connected layers with ReLU and Dropout
        x = self.relu(self.fc1(lstm_out))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)  # Final prediction
        return x


#deep learning meta model architecture
class ParallelCNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(out_features=128), #linear layer that automatically infer the input size
            nn.ReLU()
        )
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc_lstm = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128*2, num_classes)

    def forward(self, x):
        #cnn takes input of shape (batch_size, channels, seq_len)
        x_cnn = x.permute(0, 2, 1)
        out_cnn = self.cnn(x_cnn)
        # lstm takes input of shape (batch_size, seq_len, input_size)
        out_lstm, _ = self.lstm(x)
        out_lstm = self.fc_lstm(out_lstm[:, -1, :])
        out = torch.cat([out_cnn, out_lstm], dim=1)
        out = self.fc(out)
        return out


class Custom_df(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index): #this method allows to retrieve a specific sample from the dataset based on its index (the index is passed to this method)
        input = torch.tensor(np.array(self.x[index]), dtype=torch.float32) #convert np array into a tensor
        target = torch.tensor(np.array(self.y[index]), dtype=torch.float32)
        return input, target

    def __len__(self):
        return len(self.x)


class DeviceDataLoader (): #receive a dataloader and move to the correct device
  def __init__(self, dl : DataLoader, device):
    self.dl = dl
    self.device = device

  def __iter__(self):
    for batch in iter(self.dl): #iter will reset the iterator every time the __iter__ mathod gets called
      yield Weighted_Average.to_device(batch, device)

  def __len__(self):
    return len(self.dl)



class Weighted_Average ():
    def __init__(self, trained_models: list, meta_model, train_data: pd.DataFrame, test_data: pd.DataFrame, weights = None, window_size : int = 10, k = 5, temperature = 1):
        self.trained_models = trained_models
        self.meta_model = self.get_meta_model(meta_model) #accept a string as an input
        self.train_data = pd.DataFrame(train_data)
        self.test_data = pd.DataFrame(test_data)
        self.MinMaxscaler = MinMaxScaler(feature_range=(-1, 1))
        self.Robustscaler = RobustScaler()
        # self.error_scaler = MinMaxScaler(feature_range=(-1, 1))
        self.weights = weights
        self.window_size = window_size
        self.oof_predictions = []
        self.test_error_predictions = []
        self.device = self.get_device()
        self.lock = threading.Lock()
        self.ensemble_preds = None
        self.tscv = OverlappingTimeSeriesSplit(n_splits=5, window_size=self.window_size) #TimeSeriesSplit object to generate time ware time splits
        self.fist_fold_idx = None
        self.test_predictions = None
        self.k = k
        self.temperature = temperature


        #if there are deep learning models
        if any(isinstance(model, torch.nn.Module) for model in trained_models):
            self.train_loader, self.test_loader = self.data_preprocessing_deep_learning(self.train_data, self.test_data, self.MinMaxscaler, self.window_size)

        #if there are machine learning models
        if any(not isinstance(model, torch.nn.Module) for model in trained_models):
            self.X_train, self.y_train, self.X_test, self.y_test = self.data_preprocessing_machine_learning(self.train_data, self.test_data, self.Robustscaler, self.window_size)



    def feature_engineer(self, volatility, prediction):
        #target
        volatility['error_volatility'] = volatility


        #covariates
        # dataset['prediction'] = pd.DataFrame(prediction)
        # dataset['predicted volatility'] = dataset['prediction'].ewm(span=10, adjust=False).std().fillna(0)

        # for lag in range(1, 5):
        for lag in range(1, 3):
            volatility[f'lag_{lag}'] = volatility['error_volatility'].shift(lag).fillna(0)



    def get_meta_model(self, meta_model):
        try:
            if meta_model == "linear_regression":
                from sklearn.linear_model import LinearRegression
                return LinearRegression()

            elif meta_model == "ridge":
                from sklearn.linear_model import Ridge
                return Ridge()

            elif meta_model == "gradient_boosting":
                from sklearn.ensemble import GradientBoostingRegressor
                return GradientBoostingRegressor()

            elif meta_model == "random_forest":
                from sklearn.ensemble import RandomForestRegressor
                return RandomForestRegressor()

            else:
                raise ValueError(f"Unsupported meta model: {meta_model}")

        except Exception as e:
            print(f"Meta model error: {e}")
            raise e



    def sliding_windows(self, data, seq_length): #helper function to genereate sliding windows
        X, y = [], []

        for i in range(len(data) - seq_length ):
            _x = data.iloc[i:i+seq_length]
            _y = data.iloc[i + seq_length]
            X.append(_x)
            y.append(_y)

        return np.array(X), np.array(y)


    def get_device(self):
        if torch.cuda.is_available():
            device = 'cuda'
        else:
            device = 'cpu'

        return device


    @staticmethod
    def to_device(data, device):
        if isinstance(data, (list, tuple)):
            return [to_device(x, device) for x in data]
        elif isinstance(data, torch.Tensor):  # Only move tensors to the device
            return data.to(device, non_blocking=True)
        else:
            return data  # For non-tensor types (e.g., strings), return as is


    @torch.no_grad()
    def inference(self, model, data_loader):
        model.eval()  # Set the model to evaluation mode
        predictions = []
        actuals = []

        for batch in data_loader:
            input, output = batch  # Unpack your batch into input and output
            outputs = model(input) # Perform the forward pass

            # Move outputs and actuals back to CPU and append to lists
            predictions.append(outputs.cpu())
            actuals.append(output.cpu())

        # Concatenate all predictions and actuals into single tensors
        predictions = torch.cat(predictions, dim=0)
        actuals = torch.cat(actuals, dim=0)

        return predictions, actuals


    def data_preprocessing_deep_learning (self, train_data, test_data, scaler, time_window):
        #scale data
        train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data))
        test_data_scaled = pd.DataFrame(scaler.transform(test_data))

        #create sliding windows
        X_train, y_train = self.sliding_windows(train_data_scaled, time_window)
        X_test, y_test = self.sliding_windows(test_data_scaled, time_window)

        #move data to tensors
        train_df = Custom_df(X_train, y_train)
        test_df = Custom_df(X_test, y_test)

        #create dataloaders (to perform training/inference in batch)
        batch_size = 128
        num_workers = 2
        pin_memory = True if self.device == "cuda" else False
        train_loader = DataLoader(train_df, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory, persistent_workers=True)
        test_loader = DataLoader(test_df, batch_size=batch_size*2, shuffle=False, num_workers=num_workers, pin_memory=pin_memory, persistent_workers=True)

        #move to device
        train_loader = DeviceDataLoader(train_loader, self.device)
        test_loader = DeviceDataLoader(test_loader, self.device)

        return train_loader, test_loader


    def data_preprocessing_machine_learning(self, train_data, test_data, scaler, time_window):
        #convert data to numpy arrays
        if isinstance(train_data, torch.Tensor):
            train_data = train_data.numpy()
        if isinstance(test_data, torch.Tensor):
            test_data = test_data.numpy()

        #scale data

        if isinstance(train_data, pd.Series):
            train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data.values.reshape(-1,1)))
            test_data_scaled = pd.DataFrame(scaler.transform(test_data.values.reshape(-1,1)))

        else:
            train_data_scaled = pd.DataFrame(scaler.fit_transform(train_data))
            test_data_scaled = pd.DataFrame(scaler.transform(test_data))

        #create sliding windows
        X_train, y_train = self.sliding_windows(train_data_scaled, time_window)
        X_test, y_test = self.sliding_windows(test_data_scaled, time_window)

        # Flatten the sliding windows
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)

        return X_train, y_train, X_test, y_test




    def k_fold_model_prediction(self, model):
        try:
            if isinstance(model, LinearBoostRegressor): #Linear Boost model predictions

                #compute k fold out of sample predictions
                oof_predictions = np.zeros(len(self.train_data))

                with self.lock:
                    for fold, (train_idx, test_idx) in enumerate(self.tscv.split(range(len(self.train_data)))):

                        #save first fold (unused)
                        if fold == 0:
                            self.fist_fold_idx = test_idx[self.window_size]


                        #data preprocessing
                        train_df, test_df = self.train_data.iloc[train_idx], self.train_data.iloc[test_idx]

                        #X,y
                        scaler = RobustScaler()
                        X_train, y_train, X_test, y_test = self.data_preprocessing_machine_learning(train_df, test_df, scaler, self.window_size)

                        #predict
                        test_prediction = model.predict(X_test)

                        #inverse scale
                        test_prediction = scaler.inverse_transform(test_prediction.reshape(-1, 1))

                        # Store predictions in the correct indices
                        oof_predictions[test_idx[self.window_size:]] = test_prediction.flatten()


                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.train_data[self.window_size:].index, y = self.train_data[self.window_size:].squeeze(), label="Train Data")
                    plt.title('Out of sample XGBoost Predictions on Train set')
                    sns.lineplot(x= self.train_data[self.fist_fold_idx:].index, y=oof_predictions[self.fist_fold_idx:].squeeze(), label="Out of sample XGBoost predictions on Train set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    # print(self.train_data[fist_fold_idx:].shape)
                    # print(oof_predictions.shape)

                    r2 = r2_score(self.train_data[self.fist_fold_idx:].values, oof_predictions[self.fist_fold_idx:])
                    print(f"R2 score: {r2}")


                return (oof_predictions[self.fist_fold_idx:], r2)


            elif isinstance(model, torch.nn.Module):


                #compute k fold out of sample predictions
                oof_predictions = np.zeros(len(self.train_data))

                with self.lock:
                    for fold, (train_idx, test_idx) in enumerate(self.tscv.split(range(len(self.train_data)))):

                        #save first fold (unused)
                        if fold == 0:
                            self.fist_fold_idx = test_idx[self.window_size]

                        #data preprocessing
                        train_df, test_df = self.train_data.iloc[train_idx], self.train_data.iloc[test_idx]

                        #X,y
                        scaler = MinMaxScaler(feature_range=(-1, 1))
                        train_loader, test_loader = self.data_preprocessing_deep_learning(train_df, test_df, scaler, self.window_size)

                        #predict
                        test_prediction, test_actual = self.inference(model, test_loader)

                        #inverse scale
                        test_prediction = scaler.inverse_transform(test_prediction)

                        # Store predictions in the correct indices
                        oof_predictions[test_idx[self.window_size:]] = test_prediction.flatten()



                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.train_data[self.window_size:].index, y = self.train_data[self.window_size:].squeeze(), label="Train Data")
                    sns.lineplot(x= self.train_data[self.fist_fold_idx:].index, y=oof_predictions[self.fist_fold_idx:].squeeze(), label=f"Out of sample {model.__class__.__name__} predictions on Train set")
                    plt.title(f"Out of sample {model.__class__.__name__} predictions on Train set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    r2 = r2_score(self.train_data[self.fist_fold_idx:].values, oof_predictions[self.fist_fold_idx:])
                    print(f"R2 score: {r2}")


                    # Explicit cleanup
                    del train_loader, test_loader
                    torch.cuda.empty_cache()



                return (oof_predictions[self.fist_fold_idx:], r2)


            else:
                raise ValueError(f"Unsupported base model type: {type(model)}")


        except Exception as e:
            print(f"Error processing model {model}: {e}")
            raise e




    def model_prediction(self, model):
        try:
            if isinstance(model, LinearBoostRegressor): #Linear Boost model predictions

                with self.lock:
                    test_prediction = model.predict(self.X_test)

                    #inverse scale
                    test_prediction = self.Robustscaler.inverse_transform(test_prediction.reshape(-1, 1))

                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.test_data[self.window_size:].index, y = self.test_data[self.window_size:].squeeze(), label="Train Data")
                    plt.title('XGBoost Predictions on Test set')
                    sns.lineplot(x= self.test_data[self.window_size:].index, y=test_prediction.squeeze(), label="Out of sample XGBoost predictions on Train set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()


                    r2 = r2_score(self.test_data[self.window_size:].values, test_prediction)
                    print(f"R2 score: {r2}")


                return (test_prediction, r2)


            elif isinstance(model, torch.nn.Module):
                #train predictions
                with self.lock: #the lock prevents multiple threads from accessing shared data at the same time
                    test_prediction, test_actual = self.inference(model, self.test_loader)

                    # Inverse Scale
                    test_prediction = self.MinMaxscaler.inverse_transform(test_prediction)

                    #visualize predictions
                    plt.figure(figsize=(12, 6))
                    sns.lineplot(x= self.test_data[self.window_size:].index, y = self.test_data[self.window_size:].squeeze(), label="Train Data")
                    sns.lineplot(x= self.test_data[self.window_size:].index, y=test_prediction.squeeze(), label=f"{model.__class__.__name__} predictions on Test set")
                    plt.title(f"{model.__class__.__name__} predictions on Test set")
                    plt.xlabel('Date')
                    plt.ylabel('Close price')
                    plt.xticks(rotation=45)  # Rotate x-axis labels
                    plt.tight_layout() # Automatically adjusts the layout
                    plt.legend()
                    plt.show()

                    r2 = r2_score(self.test_data[self.window_size:].values, test_prediction)
                    print(f"R2 score: {r2}")

                return (test_prediction, r2)


            else:
                raise ValueError(f"Unsupported base model type: {type(model)}")


        except Exception as e:
            print(f"Error processing model {model}: {e}")
            raise e


    def predict(self):

        #k fold train predictions to infer model weights
        if (self.weights == None):
            print("k fold train predictions")
            with ThreadPoolExecutor() as executor: #it creates a pool of worker threads (the with statement ensures that the pool of threads is cleaned up automatically after the execution)
                k_fold_results = list(executor.map(self.k_fold_model_prediction, self.trained_models)) #the task each thread will execute

            r2 = np.hstack([item[1] for item in k_fold_results])
            r2 = torch.tensor(r2, dtype=torch.float32).detach()
            print('r2')
            print(r2)

            #filter the 3 best r2
            best_r2, indices = torch.topk(r2, k=self.k, largest=False, sorted=False)
            print("best k errors")
            print(best_r2)


            #normalize the errors
            std = best_r2.std(dim=0, keepdim=True) + 1e-8 #for numerical stability (to avoid division by zero)
            print("std")
            print(std[:10])

            mean = best_r2.mean(dim=0, keepdim=True)
            print("mean")
            print(mean[:10])

            normalized_r2 = (r2 - mean) / std
            print("unmasked normalized errors")
            print(normalized_r2)


            #apply mask
            mask = torch.zeros_like(r2, dtype=torch.bool)
            mask.scatter_(dim=0, index=indices, value=True)
            normalized_r2[~mask] = float('inf')
            print("normalized errors")
            print(normalized_r2)


            #apply softmax
            self.weights = F.softmax(normalized_r2 * self.temperature, dim=0)

            print('weights')
            print(self.weights)



        #retrain model on all train set
        print("Weighted prediction on test set")
        with ThreadPoolExecutor() as executor: #it creates a pool of worker threads (the with statement ensures that the pool of threads is cleaned up automatically after the execution)
            test_results = list(executor.map(self.model_prediction, self.trained_models)) #the task each thread will execute

        #stack test predictions
        self.test_predictions = np.hstack([item[0].reshape(-1, 1) for item in test_results])
        # print(self.test_predictions.shape)


        #weighted average of the predictions
        final_preds = np.average(self.test_predictions, axis=1, weights=self.weights)

        return final_preds

In [ ]:
#using the class
train_data = {}
test_data = {}

trained_models = [Linear_booster_model, CNN_model, LSTM_model, LSTM_CNN_model, parallelCNNLSTM_model]

for name , _ in datasets.items():

    # if name == 'sp500':
    #     continue

    # if name == 'eur_usd':
    #     continue

    print(f"ensembling predictions on {name} dataset:")

    meta_model = "linear_regression"
    # weights = [0.2, 0.2, 0.2, 0.2, 0.2]
    weights = None

    train_data[name] = val_residuals[name]
    train_data[name].index = pd.to_datetime(val_residuals[name].index)

    test_data[name] = test_residuals[name]
    test_data[name].index = pd.to_datetime(test_residuals[name].index)
    window_size = 10
    k = 5
    temperature = 1


    #stacking
    ensemble = Weighted_Average([trained_model[name] for trained_model in trained_models], meta_model, train_data[name], test_data[name], weights, window_size, k, temperature) #k represent the number of model to select from the batch of models
    final_preds = ensemble.predict()


    #visualize
    ensemble_preds = pd.DataFrame(final_preds, index = test_data[name][window_size:].index)

    plt.figure(figsize=(12, 6))
    plt.title(f"Weighted average ensemble predictions {name}")
    sns.lineplot(x = ensemble_preds.index, y = ensemble_preds.squeeze(), label = "ADE", color = "red")
    # sns.lineplot(x = test_data[name][window_size*2:].index, y = test_data[name][window_size*2:].squeeze(), label = "test set", color = "blue")
    sns.lineplot(x = test_data[name][window_size:].index, y = test_data[name][window_size:].squeeze(), label = "test set", color = "blue")
    plt.ylabel('Close price residuals')
    plt.xlabel('Date')
    plt.xticks(rotation=45)  # Rotate x-axis labels
    plt.tight_layout() # Automatically adjusts the layout
    plt.legend()
    plt.show()


    #compute metrics
    # mse = mean_squared_error(test_data[name][window_size*2:], ensemble_preds)
    # rmse = np.sqrt(mse)
    # mae = mean_absolute_error(test_data[name][window_size*2:], ensemble_preds)
    # r2 = r2_score(test_data[name][window_size*2:], ensemble_preds)
    # mape = mean_absolute_percentage_error(test_data[name][window_size*2:], ensemble_preds)
    # smape_value = smape(test_data[name][window_size*2:], ensemble_preds)

    mse = mean_squared_error(test_data[name][window_size:], ensemble_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_data[name][window_size:], ensemble_preds)
    r2 = r2_score(test_data[name][window_size:], ensemble_preds)
    mape = mean_absolute_percentage_error(test_data[name][window_size:], ensemble_preds)
    smape_value = smape(test_data[name][window_size:], ensemble_preds)

    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"r2:  {r2}")
    print(f"MAPE: {mape}")
    print(f"SMAPE: {smape_value}")


    #combine ensemble predictions with ARIMA model
    final_preds = SARIMA_test_predictions[name][window_size:] + ensemble_preds.iloc[:,0]

    #visualize
    plt.figure(figsize=(12, 6))
    plt.title(f"SARIMA + ensemble predictions {name}")
    sns.lineplot(x = final_preds.index, y = final_preds.squeeze(), label = "SARIMA + ensemble predictions", color = "red")
    sns.lineplot(x = Close_price[name].loc[final_preds.index].index, y = Close_price[name].loc[final_preds.index].squeeze(), label = "test set", color = "blue")
    plt.ylabel('Close price')
    plt.xlabel('Date')
    plt.xticks(rotation=45)  # Rotate x-axis labels
    plt.tight_layout() # Automatically adjusts the layout
    plt.legend()
    plt.show()


    #compute metrics
    mse = mean_squared_error(Close_price[name].loc[final_preds.index], final_preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(Close_price[name].loc[final_preds.index], final_preds)
    r2 = r2_score(Close_price[name].loc[final_preds.index], final_preds)
    mape = mean_absolute_percentage_error(Close_price[name].loc[final_preds.index], final_preds)
    smape_value = smape(Close_price[name].loc[final_preds.index], final_preds)

    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"r2:  {r2}")
    print(f"MAPE: {mape}")
    print(f"SMAPE: {smape_value}")

Output hidden; open in https://colab.research.google.com to view.